In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
cd /content/drive/MyDrive/Graduate/CS263_NLP/'[CS 263 - Fall 2024] Final Project'

/content/drive/.shortcut-targets-by-id/1OqYMWeAesj8AHJbiyPiJmxwYLnsUOIX4/[CS 263 - Fall 2024] Final Project


In [3]:
!pip install transformers
!pip install transformers accelerate

In [6]:
!pip install huggingface-hub
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: fineG

In [4]:
import os
import re
import json
from transformers import AutoTokenizer, AutoModelForCausalLM

# Initialize model and tokenizer
model_name = "meta-llama/Llama-2-7b-chat-hf"  # Replace with your model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).cuda()#, device_map="auto")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model-00001-of-00002.safetensors:   6%|6         | 608M/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [5]:
root_dir = 'MIMIC-CXR-filtered/test'
output_file = 'labels.json'

case_names = os.listdir(root_dir)
question = "Does the report mention any abnormalities, unhealthy findings, or potential health concerns? Respond with one number: 1 for 'yes' (if the report identifies any abnormality or issue) or 0 for 'no' (if the report states the findings are normal and healthy)."
results = {}

for case_name in case_names:
    print(case_name)
    json_file_path = os.path.join(root_dir, case_name, case_name + '.json')

    # Load findings from JSON file
    with open(json_file_path, 'r') as json_file:
        report = json.load(json_file)
    findings = report['FINDINGS']
    context = 'Here is the finding section of one X-ray report: ' + re.sub(r"\s+", " ", findings).strip()
    prompt = f"{context}\n\nQuestion: {question}\n\nAnswer:"
    context = re.sub(r"\s+", " ", findings).strip()
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    output = model.generate(
        inputs["input_ids"],
        max_new_tokens=10,
        temperature=0.0,
        top_p=1.0,
        do_sample=False
    )

    answer = tokenizer.decode(output[0], skip_special_tokens=True).split("Answer:")[-1].strip()
    try:
        results[case_name] = answer
    except ValueError:
        print(f"Warning: Could not parse answer '{answer}' for case '{case_name}'")
        results[case_name] = None  # Save None if the answer is not valid
    print("prompt:", prompt)
    print("Answer:", answer)
    print('\n')
with open(output_file, 'w') as outfile:
    json.dump(results, outfile, indent=4)
print(f"Results saved to {output_file}")

s58619815


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


prompt: Here is the finding section of one X-ray report: The heart is normal in size. The mediastinal and hilar contours appear within normal limits. The aortic arch shows patchy calcification. Mild subpleural thickening is unchanged at each lung apex for the most part, but there is an apparent increase in density in the left upper lung over about 1 cm region. Otherwise, the lung fields appear clear. There are no pleural effusions or pneumothorax.

Question: Does the report mention any abnormalities, unhealthy findings, or potential health concerns? Respond with one number: 1 for 'yes' (if the report identifies any abnormality or issue) or 0 for 'no' (if the report states the findings are normal and healthy).

Answer:
Answer: 1


s58971208
prompt: Here is the finding section of one X-ray report: The ET tube terminates approximately 2.9 cm from the carina. The NG tube courses below the diaphragm with the tip out of the field of view of the film. There has been interval worsening of the 

In [6]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tqdm import tqdm
pred_json = 'Results/263_fewshot_gpt-4-turbo.json'
with open(pred_json, 'r') as f:
    preds = json.load(f)
label_json = 'labels.json'
with open(label_json, 'r') as f:
    labels = json.load(f)
# question = "Are there any abnormal or unhealthy findings in this report? Respond with one number: 1 for 'yes' if any abnormality is present, or 0 for 'no' if everything is normal."
label_list = []
pred_list = []
for case_name in tqdm(case_names):
    # print(case_name)
    findings = preds[case_name].split(':')[-1]
    context = 'Here is the finding section of one X-ray report: ' + re.sub(r"\s+", " ", findings).strip()
    prompt = f"{context}\n\nQuestion: {question}\n\nAnswer:"
    context = re.sub(r"\s+", " ", findings).strip()
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    output = model.generate(
        inputs["input_ids"],
        max_new_tokens=10,
        temperature=0.0,
        top_p=1.0,
        do_sample=False
    )
    # print("prompt:", prompt)
    pred = tokenizer.decode(output[0], skip_special_tokens=True).split("Answer:")[-1].strip()
    label = labels[case_name]
    label_list.append(int(label.split('\n')[0]))
    pred_list.append(int(pred.split('\n')[0]))
    # print(label, pred)
    # print('\n')

accuracy = accuracy_score(label_list, pred_list)
precision = precision_score(label_list, pred_list)
recall = recall_score(label_list, pred_list)
f1 = f1_score(label_list, pred_list)
print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")

  0%|          | 0/53 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
100%|██████████| 53/53 [00:14<00:00,  3.60it/s]

Accuracy: 0.47
Precision: 0.55
Recall: 0.37
F1 Score: 0.44
